In this notebook we demonstrate ... with a fairness-constrained training task on the COMPAS dataset.

In the first part, we will compare two neural networks on the dataset - one with fairness constraints, one without - and compare the two.

In the second part, we will use the same setup to compare two algorithms for constrained optimization: Augmented Lagrangian and Stochastic Ghost.

In [54]:
import sys
import os
import numpy as np
import torch
from torch import tensor
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import BCELoss
from torch import nn
from torch.nn.functional import relu, sigmoid
import pandas as pd
from sklearn.metrics import roc_curve, auc, log_loss

SENS_ATTR_IND = 4
SENSITIVE_CODE_0 = 0
SENSITIVE_CODE_1 = 1

if ".." not in sys.path:
    sys.path.insert(0, "..")
from humancompatible.train.tests.utils.datasets.load_compas import load_compas
from humancompatible.train.optimize.auglagr import ALOptimizer
from humancompatible.train.optimize.stochastic_ghost import StochasticGhost
from humancompatible.train.connect.pytorch_connect import CustomNetwork, load_model

In [55]:
x_train_raw, X_train, y_train, X_val, y_val = load_compas()

File already exists. Not saving again.
File already exists. Not saving again.


c:\Users\andre\phd\humancompatible-train\examples\..\humancompatible\train\tests\utils\datasets\load_compas.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_needed['race_code'] = df_needed['race'].map(race_mapping)
c:\Users\andre\phd\humancompatible-train\examples\..\humancompatible\train\tests\utils\datasets\load_compas.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_needed['crime_code'] = pd.Categorical(df_needed['c_charge_degree']).codes
c:\Users\andre\phd\humancompatible-train\examples\..

In [56]:
ip_size = x_train_raw.shape[1]

white_idx = (x_train_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_1)
black_idx = (x_train_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_0)
x_black = X_train[black_idx, :]
y_black = y_train[black_idx]
x_white = X_train[white_idx, :]
y_white = y_train[white_idx]

x_train_raw = np.concatenate((x_train_raw[white_idx], x_train_raw[black_idx][:160]))
    
X_train = np.concatenate([x_white, x_black[:160]])
y_train = np.concatenate([y_white, y_black[:160]])

train_ds = TensorDataset(tensor(X_train[:, :ip_size], dtype=torch.float),tensor(y_train, dtype=torch.float))
train_loader = DataLoader(train_ds, batch_size=16)
    
con_ds_b = TensorDataset(tensor(x_black[:160, :ip_size], dtype=torch.float),tensor(y_black[:160], dtype=torch.float))
con_loader_b = DataLoader(con_ds_b, batch_size=16)
con_ds_w = TensorDataset(tensor(x_white[:, :ip_size], dtype=torch.float),tensor(y_white, dtype=torch.float))
con_loader_w = DataLoader(con_ds_w, batch_size=16)

## Overview of algorithms

Since we want to demonstrate the effect of constraints, we make the dataset unbalanced with respect to the sensitive attribute:

### Augmented Lagrangian

Define a simple DNN:

In [4]:
class Net(nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super().__init__()
        self.lh = nn.Linear(n_in, n_hidden)
        self.lo = nn.Linear(n_hidden, n_out)

    def forward(self, input):
        hidden = relu(self.lh(input))
        out = sigmoid(self.lo(hidden))
        return out

Just as we use ```torch.utils.data.DataLoader``` to evaluate training loss batch by batch, we use ```DataLoader```'s to evaluate constraints. Here, we take two separate instances of the ```DataLoader``` class to sample equally sized batches of white and black individuals. Subject to change.

In [ ]:
def eq_loss_constr(loss, net, c_data, loss_bound):
    w_inputs, w_labels = c_data[0]
    b_inputs, b_labels = c_data[1]
    w_outs = net(w_inputs)
    w_loss = loss(w_outs, w_labels)
    b_outs = net(b_inputs)
    b_loss = loss(b_outs, b_labels)

    return torch.abs(w_loss - b_loss) - loss_bound

con_loader = [con_loader_w, con_loader_b]

Train:

In [ ]:
loss = BCELoss()
net = Net(7, 50, 1)
loss_bound = 1e-2
sampling_int = 1
alo = ALOptimizer(net, loss, m_det = 0, m_st=1, st_constraint_fn=lambda net,
                  d: eq_loss_constr(loss, net, d, loss_bound), lr = 5e-4, t=1.5)
alo.optimize_(train_loader, con_loader, maxiter=30, epochs=5, constr_sampling_interval=sampling_int, verbose=False)

Let's check how well the trained model satisfies the constraints on training data:

In [ ]:
cval = 0
l = 0
for d in zip(*[iter(c) for c in con_loader]):
    cval += eq_loss_constr(BCELoss(), net, d, loss_bound)
    l+=1
print(l)
cval/l

10


tensor(-0.0050, grad_fn=<DivBackward0>)

In [8]:
lv = 0
for input, label in zip(X_val, y_val):
    lv += loss(net(torch.tensor(X_val, dtype=torch.float)), torch.tensor(y_val, dtype=torch.float))
lv /= len(X_val)
lv

tensor(0.6605, grad_fn=<DivBackward0>)

### Stochastic Ghost

Let's try doing the same with StochasticGhost. (LINK TO GHOST)

**Note:** A unified interface is in development, so for now, Stochastic Ghost requires a bit more work to set up.

In [ ]:
class Operations:

    # For now the input data is passed as init parameters
    def __init__(self, data, net):

        # Create a list of linear layers based on layer_sizes
        self.x_train = data[0]
        self.y_train = data[1]
        self.x_val = data[2]
        self.y_val = data[3]
        self.x_train_raw = data[4]
        self.model = net

        black_idx = (self.x_train_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_1)
        self.x_black = self.x_train[black_idx, :]
        self.y_black = self.x_train[black_idx]

        white_idx = (self.x_train_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_0)
        self.x_white = self.x_train[white_idx, :]
        self.y_white = self.x_train[white_idx]

    def obj_fun(self, params, minibatch):
        x = self.x_train
        y = self.y_train
        model = self.model
        samples = np.random.choice(len(y), minibatch, replace=False)
        fval = model.get_obj(x[samples, :], y[samples], params)
        return fval

    def obj_grad(self, params, minibatch):
        fgrad = []
        x = self.x_train
        y = self.y_train
        model = self.model
        samples = np.random.choice(len(y), minibatch, replace=False)
        fgrad = model.get_obj_grad(x[samples, :], y[samples], params)
        return fgrad

    def conf1(self, params, minibatch):
        model = self.model
        
        black_batch_idxs = np.random.choice(len(self.y_black), minibatch, replace=False)
        white_batch_idxs = np.random.choice(len(self.y_white), minibatch, replace=False)
   
        conf1 = model.get_constraint(
            self.x_black[black_batch_idxs, :], self.y_train[black_batch_idxs],
            self.x_white[white_batch_idxs, :], self.y_train[white_batch_idxs],
            params)
        return conf1
    
    def conJ1(self, params, minibatch):
        model = self.model

        black_batch_idxs = np.random.choice(len(self.y_black), minibatch, replace=False)
        white_batch_idxs = np.random.choice(len(self.y_white), minibatch, replace=False)

        conj1 = model.get_constraint_grad(
            self.x_black[black_batch_idxs, :], self.y_train[black_batch_idxs],
            self.x_white[white_batch_idxs, :], self.y_train[white_batch_idxs],
            params)
        
        return conj1

    def conf2(self, params, minibatch):
        model = self.model

        black_batch_idxs = np.random.choice(len(self.y_black), minibatch, replace=False)
        white_batch_idxs = np.random.choice(len(self.y_white), minibatch, replace=False)

        conf2 = model.get_constraint(
            self.x_black[black_batch_idxs, :], self.y_train[black_batch_idxs],
            self.x_white[white_batch_idxs, :], self.y_train[white_batch_idxs],
            params)
        
        return conf2
    
    def conJ2(self, params, minibatch):
        model = self.model

        black_batch_idxs = np.random.choice(len(self.y_black), minibatch, replace=False)
        white_batch_idxs = np.random.choice(len(self.y_white), minibatch, replace=False)

        conj2 = model.get_constraint_grad(
            self.x_black[black_batch_idxs, :], self.y_train[black_batch_idxs],
            self.x_white[white_batch_idxs, :], self.y_train[white_batch_idxs],
            params)
        
        return conj2
    
def paramvals(maxiter, beta, rho, lamb, hess, tau, mbsz, numcon, geomp, stepdecay, gammazero, zeta, N, n, lossbound, scalef):
    params = {
        'maxiter': maxiter,  # number of iterations performed
        'beta': beta,  # trust region size
        'rho': rho,  # trust region for feasibility subproblem
        'lamb': lamb,  # weight on the subfeasibility relaxation
        'hess': hess,  # method of computing the Hessian of the QP, options include 'diag' 'lbfgs' 'fisher' 'adamdiag' 'adagraddiag'
        'tau': tau,  # parameter for the hessian
        'mbsz': mbsz,  # the standard minibatch size, used for evaluating the progress of the objective and constraint
        'numcon': numcon,  # number of constraint functions
        'geomp': geomp,  # parameter for the geometric random variable defining the number of subproblem samples
        # strategy for step decrease, options include 'dimin' 'stepwise' 'slowdimin' 'constant'
        'stepdecay': stepdecay,
        'gammazero': gammazero,  # initial stepsize
        'zeta': zeta,  # parameter associated with the stepsize iteration
        'N': N,  # Train/val sample size
        'n': n,  # Total number of parameters
        'lossbound': lossbound, #Bound on constraint loss
        'scalef': scalef #Scaling factor for constraints
    }
    return params

In [10]:
data = (X_train[:, :ip_size], y_train, X_val[:, :ip_size], y_val, x_train_raw)

In [11]:
layer_sizes = [ip_size, 50, 1]
net = CustomNetwork((layer_sizes,))
operations = Operations(data, net)

Train the model:

In [ ]:
initw, num_param = net.get_trainable_params()
num_trials = min(len(y_train[((x_train_raw[:, SENS_ATTR_IND]) == SENSITIVE_CODE_1)]), len(y_train[(x_train_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_0)]))
params = paramvals(maxiter=1000, beta=10., rho=1e-3, lamb=0.5, hess='diag', tau=2., mbsz=100,
                        numcon=2, geomp=0.2, stepdecay='dimin', gammazero=0.1, zeta=0.7, N=num_trials, n=num_param, lossbound=[loss_bound, loss_bound], scalef=[1., 1.])
solver_params = {'max_iter': 400, 'eps_abs': 1e-9, 'polish': True, 'eps_prim_inf': 1e-6, 'eps_dual_inf': 1e-6}
w, iterfs, itercs = StochasticGhost(operations.obj_fun, operations.obj_grad, [operations.conf1, operations.conf2], [operations.conJ1, operations.conJ2],
                                    initw, params, solver_params=solver_params)

c:\Users\andre\miniconda3\envs\hcomp\lib\site-packages\qpsolvers\conversions\ensure_sparse_matrices.py:24: UserWarning: Converted G to scipy.sparse.csc.csc_matrix
For best performance, build G as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
c:\Users\andre\miniconda3\envs\hcomp\lib\site-packages\qpsolvers\conversions\ensure_sparse_matrices.py:24: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(


Avg iter time: 0.03790126256257185


In [ ]:
state_dict = net.state_dict()

for k, val in zip(state_dict.keys(), w):
    state_dict[k] = torch.tensor(val)

net.load_state_dict(state_dict)

<All keys matched successfully>

See how well the model satisfies the constraints on training data:

In [ ]:
cval = 0
l = 0
for d in zip(*[iter(c) for c in con_loader]):
    cval += eq_loss_constr(BCELoss(), net, d, loss_bound)
    l+=1
print(l)
cval/l

10


tensor(0.0358, grad_fn=<DivBackward0>)

Evaluate the model on validation data:

In [19]:
lv = 0
for input, label in zip(X_val, y_val):
    lv += loss(net(torch.tensor(X_val, dtype=torch.float)), torch.tensor(y_val, dtype=torch.float))
lv /= len(X_val)
lv

tensor(0.7179, grad_fn=<DivBackward0>)

## Performance comparison

In this section, we use data from experiments conducted prior to compare the two algorithms on runtime, minimization and constraint satisfaction.

In [61]:
DATASET = 'compas'
MODEL = "pytorch_connect"
MODEL_NAME = "../humancompatible/train/connect/pytorch_connect"
DIRECTORY_PATH = "../humancompatible/train/tests/utils/saved_models/" + DATASET + '/' + MODEL
FILE_EXT = '.pt'

In [62]:
directory_path = DIRECTORY_PATH

# List all the files in the directory
file_list = os.listdir(directory_path)

# Filter files to select only model files (e.g., with '.pt' extension)
model_files = [file for file in file_list if DATASET in file and file.endswith(FILE_EXT)]

# Create an empty dictionary to store loaded models
loaded_models = []

# Load each model and store it in the dictionary
for model_file in model_files:
    # Extract model name from the file name
    model_name = model_file.split('.')[0]
    
    # Load the model
    model_load = load_model(directory_path, model_file)
    
    # Add the loaded model to the dictionary
    loaded_models.append((model_file, model_load))


df_raw = pd.read_csv(f'../humancompatible/train/tests/utils/datasets/raw_data/{DATASET}/val_data_raw_'+str(DATASET)+'.csv')
df_scaled = pd.read_csv(f'../humancompatible/train/tests/utils/datasets/raw_data/{DATASET}/val_data_scaled_'+str(DATASET)+'.csv')
x_raw = df_raw.values[:,:-1]
x_scaled = df_scaled.values[:,:-1]
y = df_scaled.values[:,-1]

We can now calculate some metrics for the models on the validation set:

In [88]:
import re

results_list = []

for model_index, model_iter in enumerate(loaded_models):
    # Set the model to evaluation mode
    (model_name, model) = model_iter
    model.eval()

    predictions_0 = model.evaluate(model.to_backend(x_scaled[(x_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_0), :]))
    predictions_1 = model.evaluate(model.to_backend(x_scaled[(x_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_1), :]))

    acc_0 = np.sum(y[(x_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_0)] == (predictions_0[:,0] > 0.5)) / len(predictions_0)
    acc_1 = np.sum(y[(x_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_1)] == (predictions_1[:,0] > 0.5)) / len(predictions_1)
    l_0 = log_loss(y[(x_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_0)], predictions_0[:,0])
    l_1 = log_loss(y[(x_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_1)], predictions_1[:,0])
    
    fpr_0, tpr_0, thresholds_0 = roc_curve(y[(x_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_0)], predictions_0)
    auc_0 = auc(fpr_0, tpr_0)

    fpr_1, tpr_1, thresholds_1 = roc_curve(y[(x_raw[:, SENS_ATTR_IND] == SENSITIVE_CODE_1)], predictions_1)
    auc_1 = auc(fpr_1, tpr_1)

    auc_hm = (auc_0*auc_1)/(auc_0 + auc_1)
    
    tpr_minus_fpr_0 = tpr_0 - fpr_0
    optimal_threshold_index_0 = np.argmax(tpr_minus_fpr_0)
    optimal_threshold_0 = thresholds_0[optimal_threshold_index_0]

    tpr_minus_fpr_1 = tpr_1 - fpr_1
    optimal_threshold_index_1 = np.argmax(tpr_minus_fpr_1)
    optimal_threshold_1 = thresholds_1[optimal_threshold_index_1]

    results_list.append({'Model': str(model_name),
                                    'AUC_Sensitive_W': auc_0,
                                    'AUC_Sensitive_B': auc_1,
                                    'AUC_HM' : auc_hm,
                                    'Accuracy_W': acc_0,
                                    'Accuracy_B': acc_1,
                                    'Loss_W': l_0,
                                    'Loss_B': l_1,
                                    # 'Optimal_Threshold_0': optimal_threshold_0,
                                    # 'Optimal_Threshold_1': optimal_threshold_1
                                    })
    
res_df = pd.DataFrame(results_list)
res_df

,Model,AUC_Sensitive_W,AUC_Sensitive_B,AUC_HM,Accuracy_W,Accuracy_B,Loss_W,Loss_B
0,net_al_compas_lb0.01_s1_tr0.pt,0.671265,0.710962,0.345272,0.651316,0.589139,0.679073,0.679354
1,net_al_compas_lb0.01_s1_tr1.pt,0.655470,0.693256,0.336917,0.625000,0.625000,0.671034,0.663727
2,net_al_compas_lb0.01_s1_tr2.pt,0.682222,0.698908,0.345232,0.633224,0.626025,0.675264,0.676811
3,net_al_compas_lb0.01_s1_tr3.pt,0.684318,0.705214,0.347304,0.646382,0.633197,0.669348,0.669994
4,net_al_compas_lb0.01_s1_tr4.pt,0.696780,0.675899,0.343090,0.659539,0.621926,0.672200,0.675057
5,net_al_compas_lb0.01_sexp_tr0.pt,0.684986,0.720992,0.351264,0.684211,0.659836,0.655715,0.649615
6,net_al_compas_lb0.01_sexp_tr1.pt,0.681798,0.714508,0.348885,0.669408,0.647541,0.658832,0.655177
7,net_al_compas_lb0.01_sexp_tr2.pt,0.650944,0.714992,0.340733,0.639803,0.659836,0.651275,0.645164
8,net_al_compas_lb0.01_sexp_tr3.pt,0.667341,0.703168,0.342393,0.656250,0.631148,0.650986,0.657380
9,net_al_compas_lb0.01_sexp_tr4.pt,0.640678,0.728294,0.340841,0.646382,0.651639,0.661562,0.654274


Sort by descending constraint violation (absolute difference between loss value on subgroups on validation set)

In [64]:
res_df['loss_abs_diff'] = abs(res_df.Loss_W - res_df.Loss_B)
res_df.sort_values(by='loss_abs_diff', ascending=True)

,Model,AUC_Sensitive_W,AUC_Sensitive_B,AUC_HM,Accuracy_W,Accuracy_B,Loss_W,Loss_B,loss_abs_diff
0,net_al_compas_lb0.01_s1_tr0.pt,0.671265,0.710962,0.345272,0.651316,0.589139,0.679073,0.679354,0.000281
3,net_al_compas_lb0.01_s1_tr3.pt,0.684318,0.705214,0.347304,0.646382,0.633197,0.669348,0.669994,0.000645
2,net_al_compas_lb0.01_s1_tr2.pt,0.682222,0.698908,0.345232,0.633224,0.626025,0.675264,0.676811,0.001547
4,net_al_compas_lb0.01_s1_tr4.pt,0.696780,0.675899,0.343090,0.659539,0.621926,0.672200,0.675057,0.002858
6,net_al_compas_lb0.01_sexp_tr1.pt,0.681798,0.714508,0.348885,0.669408,0.647541,0.658832,0.655177,0.003655
15,net_ghost_compas_lb0.01_tr4.pt,0.438208,0.461370,0.224745,0.457237,0.469262,0.696595,0.700287,0.003692
5,net_al_compas_lb0.01_sexp_tr0.pt,0.684986,0.720992,0.351264,0.684211,0.659836,0.655715,0.649615,0.006100
7,net_al_compas_lb0.01_sexp_tr2.pt,0.650944,0.714992,0.340733,0.639803,0.659836,0.651275,0.645164,0.006111
8,net_al_compas_lb0.01_sexp_tr3.pt,0.667341,0.703168,0.342393,0.656250,0.631148,0.650986,0.657380,0.006395
9,net_al_compas_lb0.01_sexp_tr4.pt,0.640678,0.728294,0.340841,0.646382,0.651639,0.661562,0.654274,0.007288
